In [1]:
import pandas as pd

In [2]:
# read data from csv
data = pd.read_csv('./data/loan_data.csv')
data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [3]:
# drop id column
data.drop(['Loan_ID'],axis=1,inplace=True)

In [4]:
# data info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             601 non-null    object 
 1   Married            611 non-null    object 
 2   Dependents         599 non-null    object 
 3   Education          614 non-null    object 
 4   Self_Employed      582 non-null    object 
 5   ApplicantIncome    614 non-null    int64  
 6   CoapplicantIncome  614 non-null    float64
 7   LoanAmount         592 non-null    float64
 8   Loan_Amount_Term   600 non-null    float64
 9   Credit_History     564 non-null    float64
 10  Property_Area      614 non-null    object 
 11  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(7)
memory usage: 57.7+ KB


In [12]:
# seperate 20% sample data as unseen data, left 80% for model training 
unseen=data.sample(frac=.20,random_state=42)
train=data.drop(unseen.index)
unseen.to_csv('./data/unseen.csv',index=False)
train.to_csv('./data/train.csv',index=False)
unseen.shape,train.shape

((123, 12), (491, 12))

In [13]:
from pycaret.classification import ClassificationExperiment
s=ClassificationExperiment()

In [14]:
s.setup(train,target='Loan_Status',session_id=930)

,Description,Value
0,Session id,930
1,Target,Loan_Status
2,Target type,Binary
3,Target mapping,"N: 0, Y: 1"
4,Original data shape,"(491, 12)"
5,Transformed data shape,"(491, 17)"
6,Transformed train set shape,"(343, 17)"
7,Transformed test set shape,"(148, 17)"
8,Numeric features,5
9,Categorical features,6


In [15]:
s.compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ridge,Ridge Classifier,0.8105,0.0000,0.8105,0.8367,0.7844,0.4699,0.5347,0.0840
lda,Linear Discriminant Analysis,0.8105,0.0000,0.8105,0.8367,0.7844,0.4699,0.5347,0.0800
lr,Logistic Regression,0.8076,0.0000,0.8076,0.8313,0.7816,0.4633,0.5252,1.5450
catboost,CatBoost Classifier,0.8018,0.0000,0.8018,0.8209,0.7781,0.4552,0.5095,1.6070
rf,Random Forest Classifier,0.7987,0.0000,0.7987,0.8020,0.7801,0.4606,0.4933,0.2030
gbc,Gradient Boosting Classifier,0.7960,0.0000,0.7960,0.7921,0.7839,0.4736,0.4900,0.1690
nb,Naive Bayes,0.7931,0.0000,0.7931,0.7906,0.7758,0.4502,0.4762,0.0800
ada,Ada Boost Classifier,0.7812,0.0000,0.7812,0.7789,0.7679,0.4321,0.4521,0.1460
lightgbm,Light Gradient Boosting Machine,0.7670,0.0000,0.7670,0.7667,0.7543,0.4080,0.4267,0.1890
et,Extra Trees Classifier,0.7492,0.0000,0.7492,0.7416,0.7355,0.3586,0.3720,0.1770


RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, positive=False, random_state=930, solver='auto',
                tol=0.0001)

In [18]:
ridge=s.create_model('ridge')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8286,0.0000,0.8286,0.8629,0.8060,0.5333,0.6030
1,0.8286,0.0000,0.8286,0.8629,0.8060,0.5333,0.6030
2,0.7714,0.0000,0.7714,0.7771,0.7413,0.3778,0.4271
3,0.7941,0.0000,0.7941,0.7890,0.7908,0.4893,0.4906
4,0.7941,0.0000,0.7941,0.7951,0.7695,0.4195,0.4610
5,0.8235,0.0000,0.8235,0.8588,0.7955,0.4848,0.5657
6,0.8529,0.0000,0.8529,0.8783,0.8354,0.5854,0.6433
7,0.7647,0.0000,0.7647,0.8235,0.7031,0.2609,0.3873
8,0.8235,0.0000,0.8235,0.8588,0.7955,0.4848,0.5657


In [19]:
tuned_ridge=s.tune_model(ridge)
tuned_ridge

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8286,0.0000,0.8286,0.8629,0.8060,0.5333,0.6030
1,0.8000,0.0000,0.8000,0.8452,0.7661,0.4394,0.5306
2,0.7714,0.0000,0.7714,0.7771,0.7413,0.3778,0.4271
3,0.8235,0.0000,0.8235,0.8179,0.8173,0.5487,0.5550
4,0.7941,0.0000,0.7941,0.7951,0.7695,0.4195,0.4610
5,0.8235,0.0000,0.8235,0.8588,0.7955,0.4848,0.5657
6,0.8529,0.0000,0.8529,0.8783,0.8354,0.5854,0.6433
7,0.7647,0.0000,0.7647,0.8235,0.7031,0.2609,0.3873
8,0.8235,0.0000,0.8235,0.8588,0.7955,0.4848,0.5657


Fitting 10 folds for each of 10 candidates, totalling 100 fits


RidgeClassifier(alpha=9.18, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, positive=False, random_state=930, solver='auto',
                tol=0.0001)

In [20]:
final_ridge=s.finalize_model(tuned_ridge)
final_ridge

Pipeline(memory=Memory(location=None),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(exclude=None, include=None,
                                               transformer=LabelEncoder())),
                ('numerical_imputer',
                 TransformerWrapper(exclude=None,
                                    include=['ApplicantIncome',
                                             'CoapplicantIncome', 'LoanAmount',
                                             'Loan_Amount_Term',
                                             'Credit_History'],
                                    transformer=SimpleImputer(add_indicator=False,
                                                              copy=T...
                                    transformer=OneHotEncoder(cols=['Dependents',
                                                                    'Property_Area'],
                                                              drop_invariant=False,
                                                              handle_missing='return_nan',
                                                              handle_unknown='value',
                                                              return_df=True,
                                                              use_cat_names=True,
                                                              verbose=0))),
                ('actual_estimator',
                 RidgeClassifier(alpha=9.18, class_weight=None, copy_X=True,
                                 fit_intercept=True, max_iter=None,
                                 positive=False, random_state=930,
                                 solver='auto', tol=0.0001))],
         verbose=False)

In [21]:
unseen_X=unseen.copy().drop('Loan_Status',axis=1)
y=unseen['Loan_Status']

prediction=s.predict_model(final_ridge,data=unseen_X)

In [22]:
from sklearn.metrics import classification_report

y_pred=prediction['prediction_label']
print(classification_report(y,y_pred))

              precision    recall  f1-score   support

           N       0.95      0.42      0.58        43
           Y       0.76      0.99      0.86        80

    accuracy                           0.79       123
   macro avg       0.85      0.70      0.72       123
weighted avg       0.83      0.79      0.76       123



In [1]:
# For the 'N' class, the precision is 0.95, indicating that 95% of instances predicted as 'N' are indeed 'N'. For the 'Y' class, the precision is 0.76, meaning that 76% of instances predicted as 'Y' are actually 'Y'.
# For the 'N' class, the recall is 0.42, indicating that the model successfully predicts 42% of actual 'N' instances. For the 'Y' class, the recall is 0.99, meaning that the model correctly predicts 99% of actual 'Y' instances.
# The F1-score for the 'N' class is 0.58, and for the 'Y' class is 0.86.
# There are 43 instances of 'N' and 80 instances of 'Y'.
# The overall accuracy is 0.79, indicating that 79% of the predictions are correct.
# Overall, the model performs well in predicting the 'Y' class but performs relatively poorly in predicting the 'N' class.